# Analisys

In [1]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Ruta del archivo JSON
file_path = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_automation.json"

# Importar los datos
with open(file_path, 'r') as file:
    relay_data = json.load(file)

# Extraer los datos de Ishc para los relés principales y de respaldo
main_relay_data = []
backup_relay_data = []

for entry in relay_data:
    scenario_id = entry["scenario_id"]
    fault = entry["fault"]
    
    # Datos del relé principal
    main_relay = entry["main_relay"]
    main_relay_data.append({
        "scenario_id": scenario_id,
        "fault": fault,
        "relay": main_relay["relay"],
        "line": main_relay.get("line", "N/A"),  # Usa get para manejar posibles casos donde line no exista
        "Ishc": main_relay["Ishc"],
        "relay_type": "Main"
    })
    
    # Datos del relé de respaldo
    backup_relay = entry["backup_relay"]
    backup_relay_data.append({
        "scenario_id": scenario_id,
        "fault": fault,
        "relay": backup_relay["relay"],
        "line": backup_relay.get("line", "N/A"),
        "Ishc": backup_relay["Ishc"],
        "relay_type": "Backup"
    })

# Crear DataFrames
main_df = pd.DataFrame(main_relay_data)
backup_df = pd.DataFrame(backup_relay_data)
combined_df = pd.concat([main_df, backup_df])

# 1. Distribución de Ishc por tipo de relé (principal vs respaldo)
fig1 = px.box(combined_df, x="relay_type", y="Ishc", color="relay_type",
             title="Distribución de Ishc por Tipo de Relé",
             labels={"relay_type": "Tipo de Relé", "Ishc": "Valor de Ishc"},
             color_discrete_map={"Main": "blue", "Backup": "red"})

# 2. Histograma de valores Ishc para relés principales y de respaldo
fig2 = px.histogram(combined_df, x="Ishc", color="relay_type", barmode="overlay",
                   title="Histograma de Valores Ishc por Tipo de Relé",
                   labels={"Ishc": "Valor de Ishc", "count": "Frecuencia", "relay_type": "Tipo de Relé"},
                   color_discrete_map={"Main": "blue", "Backup": "red"})

# 3. Scatter plot de Ishc para relé principal vs relé de respaldo por cada par
pair_df = pd.DataFrame({
    "scenario_id": main_df["scenario_id"],
    "fault": main_df["fault"],
    "main_relay": main_df["relay"],
    "main_line": main_df["line"],
    "main_Ishc": main_df["Ishc"],
    "backup_relay": backup_df["relay"],
    "backup_line": backup_df["line"],
    "backup_Ishc": backup_df["Ishc"]
})

fig3 = px.scatter(pair_df, x="main_Ishc", y="backup_Ishc", 
                 hover_data=["scenario_id", "main_relay", "backup_relay", "main_line", "backup_line"],
                 title="Relación entre Ishc de Relés Principales y de Respaldo",
                 labels={"main_Ishc": "Ishc Relé Principal", "backup_Ishc": "Ishc Relé de Respaldo"})

# 4. Compara Ishc a través de diferentes escenarios (tomar los primeros 10 escenarios para mayor claridad)
unique_scenarios = combined_df["scenario_id"].unique()[:10]
scenario_df = combined_df[combined_df["scenario_id"].isin(unique_scenarios)]

fig4 = px.box(scenario_df, x="scenario_id", y="Ishc", color="relay_type",
             title="Distribución de Ishc por Escenario y Tipo de Relé (10 primeros escenarios)",
             labels={"scenario_id": "ID de Escenario", "Ishc": "Valor de Ishc", "relay_type": "Tipo de Relé"},
             color_discrete_map={"Main": "blue", "Backup": "red"})

# 5. Mapa de calor para la relación Ishc por relé y escenario (muestra de relés)
# Primero, preparamos los datos para el mapa de calor
top_relays = combined_df["relay"].value_counts().head(15).index.tolist()
heatmap_df = combined_df[combined_df["relay"].isin(top_relays)]
pivot_df = pd.pivot_table(heatmap_df, values="Ishc", index="relay", columns="scenario_id", aggfunc="mean")

# Mostrar solo los primeros 10 escenarios para mayor claridad
pivot_df = pivot_df.iloc[:, :10]

fig5 = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    colorscale='Viridis',
    colorbar=dict(title="Ishc")
))

fig5.update_layout(
    title="Mapa de Calor de Valores Ishc por Relé y Escenario",
    xaxis_title="ID de Escenario",
    yaxis_title="Relé"
)

# Mostrar todas las figuras
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()

# 6. Crear un gráfico de dispersión 3D para visualizar la relación entre escenario, relé y valor Ishc
# Para limitar el tamaño, tomaremos una muestra
sample_df = combined_df.sample(min(1000, len(combined_df)))

fig6 = px.scatter_3d(sample_df, x="scenario_id", y="relay", z="Ishc", color="relay_type",
                    title="Relación 3D entre Escenario, Relé y Valor Ishc (Muestra)",
                    labels={"scenario_id": "ID de Escenario", "relay": "Relé", "Ishc": "Valor de Ishc"},
                    color_discrete_map={"Main": "blue", "Backup": "red"})
fig6.show()

# 7. Comparación de la media de Ishc de los relés principales y de respaldo para cada escenario
scenario_stats = combined_df.groupby(["scenario_id", "relay_type"])["Ishc"].mean().reset_index()
scenario_pivot = pd.pivot_table(scenario_stats, values="Ishc", index="scenario_id", columns="relay_type")

fig7 = make_subplots(specs=[[{"secondary_y": True}]])

fig7.add_trace(
    go.Scatter(x=scenario_pivot.index, y=scenario_pivot["Main"], name="Main Relay",
              line=dict(color="blue")),
    secondary_y=False
)

fig7.add_trace(
    go.Scatter(x=scenario_pivot.index, y=scenario_pivot["Backup"], name="Backup Relay",
              line=dict(color="red")),
    secondary_y=True
)

fig7.update_layout(
    title_text="Media de Ishc por Tipo de Relé a través de Escenarios",
    xaxis_title="ID de Escenario"
)

fig7.update_yaxes(title_text="Ishc Relé Principal", secondary_y=False)
fig7.update_yaxes(title_text="Ishc Relé de Respaldo", secondary_y=True)

fig7.show()
